In [ ]:
#import des librairies
from collatex import *
from lxml import etree
from xml.etree import ElementTree as et
import os
import json

In [ ]:
#on parse les documents
A1 = etree.parse('textes/AsansLemmes.xml')
B1 = etree.parse('textes/BsansLemmes.xml')
C1 = etree.parse('textes/CsansLemmes.xml')

In [ ]:
#défintion de la fonction qui permet la transformation vers du json, puisque collatex doit prendre du json
#on utilise une feuille de transformation
def XMLtoJson(id,xmlInput):
    #défition de witness comme un dictionnaire
    witness = {}
    #on donne la première valeur de l'id
    witness['id'] = id
    #on transforme le document XML : d'abord on tokénise, puis on transforme en json
    tokenization = etree.XML("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output method="xml" indent="yes" encoding="UTF-8" omit-xml-declaration="yes"/>
   
    <xsl:template match="/*">
        <xsl:copy>
            <xsl:apply-templates select="@*"/>
            <xsl:apply-templates/>
        </xsl:copy>
    </xsl:template>
    
    <xsl:template match="text()">
        <xsl:call-template name="tokenize">
            <xsl:with-param name="param1" select="." />
        </xsl:call-template>
    </xsl:template>
    
    <xsl:template name="tokenize">
        <xsl:param name="param1" />
        <xsl:variable name="Pretoken" select="normalize-space(
            substring-before(concat( $param1, ','), ','))" />
        <xsl:variable name="Pretoken2" select="normalize-space(
            substring-before(concat( $Pretoken, '.'), '.'))" />
        <xsl:variable name="token" select="normalize-space(
            substring-before(concat($Pretoken2, ' '), ' ')
            )" />

        <xsl:if test="$token">
        <xsl:choose>
            <xsl:when test="not(contains($token,  '’'))">
                <w>
                    <xsl:value-of select="$token" />
                </w>
                
            </xsl:when>
            <xsl:when test="contains($token,  '’')">
                <w>
                    <xsl:value-of select="substring-before($token,'’')"/>
                </w>
                <w>
                    <xsl:value-of select="substring-after($token,'’')"/>
                </w>
            </xsl:when>
        </xsl:choose>
            <xsl:call-template name="tokenize">
                <xsl:with-param name="param1" select="substring-after($param1, ' ')" />
            </xsl:call-template>
        </xsl:if>
         
    </xsl:template>     
    
</xsl:stylesheet>
    
""")
    transformTok = etree.XSLT(tokenization)
                           
    XSLtoJSON = etree.XML('''
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
    xmlns:xs="http://www.w3.org/2001/XMLSchema"
    exclude-result-prefixes="xs"
    version="1.0">

    <xsl:output method="text"/>

    <xsl:template match="/">
        <xsl:apply-templates/>
    </xsl:template>

    <xsl:template match="w">
        <xsl:text>{"t": "</xsl:text>
        <xsl:apply-templates/>
        <xsl:text>"}</xsl:text>
        <xsl:if test="following::w">
            <xsl:text>, </xsl:text>
        </xsl:if>
    </xsl:template>
</xsl:stylesheet>
    ''')
    #on applique l'XSL
    monXSL = etree.XSLT(XSLtoJSON)
    #on charge le résultat de l'XSL dans la valeur de l'entrée du dicitonnaire "tokens"
    witness['tokens'] = json.loads( '[' +str(monXSL(transformTok(xmlInput))) +']')
    return witness

In [ ]:
#le dictionnaire niveau général
json_input = {}
#la première entrée qui contient les witnesses
json_input['witnesses'] = []
#application de la fonction de transformation sur A puis sur B puis sur C
json_input['witnesses'].append(XMLtoJson('A',A1))
json_input['witnesses'].append(XMLtoJson('B',B1))
json_input['witnesses'].append(XMLtoJson('C',C1))
print(json_input)
#on a bien à  le json créé !      

In [ ]:
#collation
coll = collate(json_input, output="table",segmentation=False)
coll

In [ ]:
#définition de la fonction notre export
def notre_export_xml(table):
    readings = []
    for column in table.columns:
        #création élément app
        app = et.Element('app')
        for key, value in sorted(column.tokens_per_witness.items()):
            #création d'élément rdg
            child = et.Element('rdg')
            child.attrib['wit'] = "#" + key
            child.text = "".join(str(item.token_data["t"]) for item in value)
            app.append(child)
        result = et.tostring(app, encoding="unicode")
        readings.append(result)
    return "".join(readings)

In [ ]:
collResult = notre_export_xml(coll)

In [ ]:
docCollResult = "<text>" + ''.join(collResult) + "</text>"

In [ ]:
docCollResult

In [ ]:
#on écrit le résultat (une simple chaîne de caractères) dans notre document XML
with open("resultat_collation_depuisXML3SansL.xml", "w") as text_file:
    text_file.write(docCollResult)   